In [1]:
import json, pandas as pd, numpy as np
from datetime import datetime
from conceptual_engineering_toolkit import Concept, Entity
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.document_loaders import WikipediaLoader

In [2]:
SAMPLE_SIZE = 4
females = np.random.choice(json.load(open('data/females-2023-03-15.json', 'r')), SAMPLE_SIZE)
males = np.random.choice(json.load(open('data/males-2023-03-15.json', 'r')), SAMPLE_SIZE)
non_binaries = np.random.choice(json.load(open('data/non-binaries-2023-03-15.json', 'r')), SAMPLE_SIZE)
trans_men = np.random.choice(json.load(open('data/trans-men-2023-03-15.json', 'r')), SAMPLE_SIZE)
trans_women = np.random.choice(json.load(open('data/trans-women-2023-03-15.json', 'r')), SAMPLE_SIZE)
sample = np.concatenate((females, trans_women, non_binaries, trans_men, males))

In [3]:
for entity in sample:
    entity["description"] = WikipediaLoader(query=entity["name"], load_max_docs=1).load()[0].page_content[:1200]

In [4]:
entities = pd.DataFrame.from_records(sample)
woman_conditions = [
    ((entities.gender == 'female') | (entities.gender == 'trans woman')),
    ((entities.gender == 'non-binary') | (entities.gender == 'male') | (entities.gender == 'trans man'))
]
woman_categories = ['true', 'false']
entities['woman'] = np.select(woman_conditions, woman_categories)
entities

,item,name,article,gender,description,woman
0,http://www.wikidata.org/entity/Q7429,Nina Andreyeva,https://en.wikipedia.org/wiki/Nina_Andreyeva,female,Nina Alexandrovna Andreyeva (Russian: Нина Але...,true
1,http://www.wikidata.org/entity/Q6399,Anita Włodarczyk,https://en.wikipedia.org/wiki/Anita_W%C5%82oda...,female,Anita Włodarczyk (Polish pronunciation: [aˈɲit...,true
2,http://www.wikidata.org/entity/Q7510,Jeehiun Lee,https://en.wikipedia.org/wiki/Jeehiun_Lee,female,Jeehiun Katherine Lee is an organic chemist an...,true
3,http://www.wikidata.org/entity/Q11630,Jane Hillston,https://en.wikipedia.org/wiki/Jane_Hillston,female,Jane Elizabeth Hillston (born 1963) is a Brit...,true
4,http://www.wikidata.org/entity/Q5085836,Charlotte Bach,https://en.wikipedia.org/wiki/Charlotte_Bach,trans woman,Charlotte Bach (born Karoly Hajdu; 1920–1981) ...,true
5,http://www.wikidata.org/entity/Q503007,Andreja Pejić,https://en.wikipedia.org/wiki/Andreja_Peji%C4%87,trans woman,"Andreja Pejić (; formerly Andrej Pejić, born 2...",true
6,http://www.wikidata.org/entity/Q7301651,Rebecca Allison,https://en.wikipedia.org/wiki/Rebecca_Allison,trans woman,"Rebecca Anne ""Becky"" Allison (born December 21...",true
7,http://www.wikidata.org/entity/Q113644651,Negha Shahin,https://en.wikipedia.org/wiki/Negha_Shahin,trans woman,Negha Shahin (born 1992/1993) is an Indian act...,true
8,http://www.wikidata.org/entity/Q5792770,Hugh Sheridan,https://en.wikipedia.org/wiki/Hugh_Sheridan,non-binary,Hugh Sheridan (born 30 June 1985) is an Austra...,false
9,http://www.wikidata.org/entity/Q4979160,Bruno Capinan,https://en.wikipedia.org/wiki/Bruno_Capinan,non-binary,Bruno Capinan ( born 18 October 1984) is a Bra...,false


In [5]:
oed_woman = Concept("oed_woman", "woman", "An adult female human being. The counterpart of man (see man, n.¹ II.4.)")

In [6]:
haslanger_woman = Concept("haslanger_woman", "woman", """S is a woman iff (i) S is regularly and for the most part observed or imagined to have 
certain bodily features presumed to be evidence of a female’s biological role in reproduction; 
(ii) that S has these features marks S within the dominant ideology of S’s society as someone who 
ought to occupy certain kinds of social position that are in fact subordinate (and so motivates 
and justifies S’s occupying such a position); and (iii) the fact that S satisfies (I) and (ii) plays 
a role in S’s systematic subordination, that is, along some dimension, S’s social position is oppressive, 
and S’s satisfying (i) and (ii) plays a role in that dimension of subordination""")

In [7]:
homosaurus_woman = Concept("homosaurus_woman", "woman", """Adults who self-identify as women and understand their gender in terms of Western conceptions of 
womanness, femaleness, and/or femininity. The term has typically been defined as adult female humans, though not all women identify with 
the term "female" depending on the context in which it is used.""")

In [8]:
oed_classifications = pd.DataFrame( 
    [ oed_woman.classify_entity(Entity(entity["item"], entity["name"], entity["description"])) for entity in sample ], 
    columns = [ 'entity' , 'classification', 'rationale' ] 
    )
oed_classifications["classification"] = oed_classifications["classification"].str.lower()

In [9]:
oed_results = pd.concat([entities[["name", "gender", "woman"]], oed_classifications[["classification", "rationale"]]], axis=1)
results_styler = oed_results.style.set_properties(**{"text-align": "left", "overflow-wrap": "break-word"})
results_styler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]) 

,name,gender,woman,classification,rationale
0,Nina Andreyeva,female,true,true,"Nina Andreyeva can be classified as a woman based on the provided definition of a woman as an adult female human being. The description of Nina Andreyeva refers to her using female pronouns such as ""she"" and ""her"", indicating that she is female. Additionally, the description provides information about her adult activities such as her career as a chemist, teacher, author, political activist, and social critic, which suggests that she is an adult. Therefore, given that she is an adult female human being, Nina Andreyeva fits the definition of a woman."
1,Anita Włodarczyk,female,true,true,"The definition of a woman is an adult female human being. Anita Włodarczyk is described as a female in the text, specifically in the context of her achievements in women's hammer throw. She is also an adult, as indicated by her birth date (8 August 1985) and her participation in adult-level sports competitions. Therefore, based on the provided definition and the information about Anita Włodarczyk, it can be argued that Anita Włodarczyk is a woman."
2,Jeehiun Lee,female,true,true,"The definition of a woman is an adult female human being. The description of Jeehiun Katherine Lee identifies her as a female, as indicated by the use of the female pronoun ""she"". Additionally, she is an adult, as evidenced by her professional accomplishments such as being a professor and running a research lab. Therefore, based on the provided definition, Jeehiun Lee can be identified as a woman."
3,Jane Hillston,female,true,true,"The definition provided for the concept of a woman is ""an adult female human being."" The entity, Jane Hillston, is described as a British computer scientist and professor, and there is no information contradicting the assertion that she is an adult or a human being. Furthermore, Jane is traditionally a female name, and she is referred to with female pronouns (""she"", ""her"") throughout the description. Therefore, based on the provided definition and the information about Jane Hillston, it can be argued that Jane Hillston is a woman."
4,Charlotte Bach,trans woman,true,true,"The text refers to Charlotte Bach using female pronouns such as ""she"" and ""her"", indicating that she is a female. Additionally, the text mentions that she was a wife, a role typically associated with women. Therefore, based on the definition of a woman as an adult female human being, we can assert that Charlotte Bach is a woman."
5,Andreja Pejić,trans woman,true,true,"The definition provided for the concept of a woman is ""an adult female human being."" According to the description provided, Andreja Pejić came out as a trans woman in 2013. This indicates that she identifies as a female, and since she is an adult human being, she fits the definition of a woman. Therefore, Andreja Pejić is a woman."
6,Rebecca Allison,trans woman,true,true,"Rebecca Allison is a woman because the text states that she transitioned in 1993. This means she changed her gender to female, which is the definition of a woman as an adult female human being. Furthermore, she is referred to with female pronouns throughout the text, reinforcing her identity as a woman."
7,Negha Shahin,trans woman,true,true,"Negha Shahin is identified as a woman based on the information provided. She is an adult, as indicated by her birth year (1992/1993) and her activities such as working as an actress, mental health counsellor, model, and television anchor. She is also identified as female, as she came out as a transgender woman and has been recognized as such in her professional life, including winning an award as the first transgender woman at the Kerala State Film Awards. Therefore, based on the definition of a woman as an adult female human being, Negha Shahin can be classified as a woman."
8,Hugh Sheridan,non-binary,false,false,"Based on the provided definition and information, there is no argument to support the assertion th

In [10]:
haslanger_classifications = pd.DataFrame( 
    [ haslanger_woman.classify_entity(Entity(entity["item"], entity["name"], entity["description"])) for entity in sample ], 
    columns = [ 'entity' , 'classification', 'rationale' ] 
    )
haslanger_classifications["classification"] = haslanger_classifications["classification"].str.lower()

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-4kh1r87IQ1kkVxtlLRc4XifU on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-4kh1r87IQ1kkVxtlLRc4XifU on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-4kh1r87IQ1kkVxtlLRc4XifU on tokens p

In [11]:
haslanger_results = pd.concat([entities[["name", "gender", "woman"]], haslanger_classifications[["classification", "rationale"]]], axis=1)
results_styler = haslanger_results.style.set_properties(**{"text-align": "left", "overflow-wrap": "break-word"})
results_styler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]) 

,name,gender,woman,classification,rationale
0,Nina Andreyeva,female,true,true,"Based on the provided definition, Nina Andreyeva can be considered a woman. The text does not explicitly state that she has the bodily features associated with the female biological role in reproduction, but it is implied by her female name and the use of female pronouns. Furthermore, as a Soviet and Russian chemist, teacher, author, political activist, and social critic, she occupied social positions that, within the dominant ideology of her society, could be considered subordinate. This is especially evident in her role as a critic and activist, where she defended traditional Soviet principles and criticized the leadership of her time. This could be seen as a form of systematic subordination, as her views were not in line with those of the ruling party. Therefore, based on the provided definition, Nina Andreyeva can be considered a woman."
1,Anita Włodarczyk,female,true,true,"Anita Włodarczyk can be considered a woman based on the provided definition. Firstly, she is observed to have certain bodily features presumed to be evidence of a female’s biological role in reproduction. This is inferred from the fact that she competes in women's sports events, such as the women's hammer throw, and holds the women's world record in this discipline. Secondly, she is marked within the dominant ideology of her society as someone who ought to occupy certain kinds of social position that are in fact subordinate. This can be inferred from the fact that she competes in a sport that is gender-segregated, which is a reflection of societal norms and expectations about the roles and abilities of different genders. Thirdly, the fact that she satisfies the first two conditions plays a role in her systematic subordination. This is evident in the fact that she competes in a sport where women's achievements are often compared to and valued less than men's achievements, reflecting a dimension of subordination. Therefore, based on the provided definition, Anita Włodarczyk can be considered a woman."
2,Jeehiun Lee,female,true,false,"The text does not provide explicit information about Jeehiun Lee's biological features or the social position she occupies in her society. Therefore, based on the given definition and the information provided, it is not possible to definitively argue that Jeehiun Lee is a woman."
3,Jane Hillston,female,true,false,"The text does not provide explicit information about Jane Hillston's biological features or the societal expectations and positions she occupies due to these features. Therefore, based on the given definition and the provided information, it is not possible to definitively argue that Jane Hillston is a woman."
4,Charlotte Bach,trans woman,true,true,"The text provides evidence that Charlotte Bach, originally born as Karoly Hajdu, later adopted the persona of a woman. This can be inferred from the use of the pronoun 'she' and the fact that she had a wife, which suggests that she was observed or imagined to have certain bodily features presumed to be evidence of a female’s biological role in reproduction (i). Furthermore, the text mentions that she occupied certain social positions, such as a writer and society hypnotherapist, which could be interpreted as traditionally female roles in her society, marking her within the dominant ideology of her society as someone who ought to occupy certain kinds of social position that are in fact subordinate (ii). Lastly, the fact that she was sentenced to gaol for acquiring a loan whilst bankrupt could be seen as a form of systematic subordination, indicating that her social position was oppressive, and her satisfying (i) and (ii) played a role in that dimension of subordination (iii). Therefore, according to the provided definition, Charlotte Bach can be considered a woman."
5,Andreja Pejić,trans woman,true,true,"According to the provided definition, Andreja Pejić can be considered a woman. Firstly, sin

In [12]:
homosaurus_classifications = pd.DataFrame( 
    [ homosaurus_woman.classify_entity(Entity(entity["item"], entity["name"], entity["description"])) for entity in sample ], 
    columns = [ 'entity' , 'classification', 'rationale' ] 
    )
homosaurus_classifications["classification"] = homosaurus_classifications["classification"].str.lower()

In [13]:
homosaurus_results = pd.concat([entities[["name", "gender", "woman"]], homosaurus_classifications[["classification", "rationale"]]], axis=1)
results_styler = homosaurus_results.style.set_properties(**{"text-align": "left", "overflow-wrap": "break-word"})
results_styler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]) 

,name,gender,woman,classification,rationale
0,Nina Andreyeva,female,true,true,"Nina Andreyeva can be identified as a woman based on the provided definition. The definition states that a woman is an adult who self-identifies as a woman and understands their gender in terms of Western conceptions of womanness, femaleness, and/or femininity. While the text does not explicitly state that Andreyeva self-identifies as a woman, it uses female pronouns (""she"") to refer to her, which suggests that she identifies as a woman. Furthermore, the definition notes that the term ""woman"" has typically been defined as adult female humans. Andreyeva, being an adult human who is referred to with female pronouns, fits this description. Therefore, based on the provided definition, it can be argued that Nina Andreyeva is a woman."
1,Anita Włodarczyk,female,true,true,"The text refers to Anita Włodarczyk using female pronouns such as ""she"" and ""her"", which indicates that she identifies as a woman. Furthermore, she is described as the first woman in history to throw the hammer over 80 m and holds the women's world record. This suggests that she understands her gender in terms of Western conceptions of womanness, femaleness, and/or femininity. Therefore, based on the provided definition, Anita Włodarczyk can be classified as a woman."
2,Jeehiun Lee,female,true,true,"The text identifies Jeehiun Lee with female pronouns such as ""she"" and ""her"", which suggests that she self-identifies as a woman. Furthermore, there is no information provided that would contradict the assertion that Jeehiun Lee understands her gender in terms of Western conceptions of womanness, femaleness, and/or femininity. Therefore, based on the provided definition and the information given about Jeehiun Lee, it can be argued that Jeehiun Lee is a woman."
3,Jane Hillston,female,true,true,"The text refers to Jane Hillston using female pronouns such as ""she"" and ""her"", which indicates that she self-identifies as a woman. Furthermore, she is an adult, which aligns with the definition of a woman as an adult who self-identifies as a woman. Therefore, based on the provided definition and the information about Jane Hillston, it can be argued that Jane Hillston is a woman."
4,Charlotte Bach,trans woman,true,true,"Charlotte Bach, born Karoly Hajdu, is a woman based on the provided definition. The definition states that a woman is an adult who self-identifies as a woman and understands their gender in terms of Western conceptions of womanness, femaleness, and/or femininity. Charlotte Bach, despite being born as Karoly Hajdu, adopted the persona of a woman and lived her life as such. This indicates that she self-identified as a woman. Furthermore, she operated in Western society, suggesting that she understood her gender in terms of Western conceptions of womanness. Therefore, based on the provided definition, it can be argued that Charlotte Bach is a woman."
5,Andreja Pejić,trans woman,true,true,"According to the provided definition, a woman is an adult who self-identifies as a woman and understands their gender in terms of Western conceptions of womanness, femaleness, and/or femininity. The entity, Andreja Pejić, is described as having come out as a trans woman in 2013. This indicates that she self-identifies as a woman, aligning with the given definition. Therefore, based on this information, it can be argued that Andreja Pejić is a woman."
6,Rebecca Allison,trans woman,true,true,"Rebecca Allison can be considered a woman based on the provided definition. The definition states that a woman is an adult who self-identifies as a woman and understands their gender in terms of Western conceptions of womanness, femaleness, and/or femininity. It is mentioned in the description that Rebecca Allison is a transgender activist who transitioned in 1993. This indicates that she self-identifies as a woman, aligning with the definition provided. Therefore, based on the information given, Rebecc

In [14]:
def evaluation_record(concept, data):
    return {
        "created": datetime.now().isoformat(),
        "concept": concept.to_json(),
        "data": data.to_dict('records'),
    }

In [15]:
oed_evaluation_record = evaluation_record(oed_woman, pd.concat([entities, oed_classifications], axis=1))
haslanger_evaluation_record = evaluation_record(haslanger_woman, pd.concat([entities, haslanger_classifications], axis=1))
homosaurus_evaluation_record = evaluation_record(homosaurus_woman, pd.concat([entities, homosaurus_classifications], axis=1))
json.dump(
    oed_evaluation_record, 
    open(f'data/{oed_evaluation_record["concept"]["model_name"]}_{oed_evaluation_record["concept"]["id"]}_{oed_evaluation_record["created"]}.json', 'w')
)
json.dump(
    haslanger_evaluation_record, 
    open(f'data/{haslanger_evaluation_record["concept"]["model_name"]}_{haslanger_evaluation_record["concept"]["id"]}_{haslanger_evaluation_record["created"]}.json', 'w')
)
json.dump(
    homosaurus_evaluation_record, 
    open(f'data/{homosaurus_evaluation_record["concept"]["model_name"]}_{homosaurus_evaluation_record["concept"]["id"]}_{homosaurus_evaluation_record["created"]}.json', 'w')
)